In [ ]:
export cwd=$(pwd)

In [ ]:
echo $cwd

In [ ]:
export SRR_RUN_TABLE=$cwd/SraRunTable.txt

In [ ]:
cd $cwd
mkdir run
cd run

In [ ]:
rm -rf *

In [ ]:
python3 $cwd/parse_sra_runtable.py $SRR_RUN_TABLE

In [ ]:
for EXP in ./*/
do
    cd $EXP
    sh $cwd/align.sh
    cd ..
done